In [1]:
import qnexus as qnx
import numpy as np

qnx.login()

🌐 Browser log in initiated.


╭────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                        │
│         Confirm that the browser shows the following code and click 'allow device':    │
│                                                                                        │
│                                      Yg4wN2                                            │
│                                                                                        │
╰────────────────────────────────────────────────────────────────────────────────────────╯

Browser didn't open automatically? Use this link: https://nexus.quantinuum.com/auth/device/browser?otp=Yg4wN29OMFN0UtIs_sx1vNVfFkPoovF3V21oXVIrl7460mZ5PHeEIkfre_R2KS-yKwJveJTKLYkIqIIPyHZCnQ
✅ Successfully logged in as j.montanez-barrera@fz-juelich.de using the browser.


In [5]:
import qnexus as qnx
import uuid

unique_suffix = uuid.uuid1()

project = qnx.projects.get_or_create("Helios-Samples")
qnx.context.set_active_project(project)

In [15]:
from guppylang import guppy
from guppylang.std.builtins import result
from guppylang.std.quantum import qubit, toffoli, s, measure
from guppylang.std.quantum.functional import h


@guppy
def repeat_until_success() -> None:
    """
    Repeat-until-success circuit for Rz(acos(3/5))
    from Nielsen and Chuang, Fig. 4.17.
    """
    q = qubit()
    for i in range(5):
        a, b = h(qubit()), h(qubit())
        toffoli(a, b, q)
        s(q)
        toffoli(a, b, q)
        if not (measure(h(a)) | measure(h(b))):
            result("rus_attempts", i)
    result("q", measure(q))


hugr_binary = repeat_until_success.compile()

In [ ]:
from guppylang import guppy
from guppylang.std.builtins import result
from guppylang.std.quantum import qubit, toffoli, s, measure, crz, rx
from guppylang.std.quantum.functional import h
import guppylang
guppylang.enable_experimental_features()

nq = 5
problems = np.load("./Data/WMC_FC.npy", allow_pickle=True).item()
G = problems[nq]["G"]


@guppy
def repeat_until_success() -> None:
    """
    Repeat-until-success circuit for Rz(acos(3/5))
    from Nielsen and Chuang, Fig. 4.17.
    """
    nq = 5
    delta = 0.3
    p = 10
    edges = [(0, 1),
            (0, 2),
            (0, 3),
            (0, 4),
            (1, 2),
            (1, 3),
            (1, 4),
            (2, 3),
            (2, 4),
            (3, 4)]
    
    betas = [i * delta / p for i in range(p+1, 1, -1)]
    gammas = [i * delta / p for i in range(1, p+1)]
    qs = [h(qubit()) for _ in range(nq)]
    ps = len(gammas)
    for p_i in range(ps):
        for i, j in edges:
            crz(qs[i], qs[j], 2 * gammas[p_i] * 1)
        for i in range(nq):
            rx(qs[i], - 2 * betas)
    [result("q", measure(qs[i])) for i in range(nq)]


hugr_binary = repeat_until_success.compile()


Error: Type mismatch (at <In[30]>:39:29)
   | 
37 |     for p_i in range(ps):
38 |         for i, j in edges:
39 |             crz(qs[i], qs[j],2 * gammas[p_i] * 1)
   |                              ^^^^^^^^^^^^^^^^^^^ Expected argument of type `angle`, got `float`

Guppy compilation failed due to 1 previous error


In [24]:
G = problems[nq]["G"]
list(G.edges)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [13]:
ref_hugr = qnx.hugr.upload(
    hugr_binary, 
    name=f"repeat-until-success-{unique_suffix}"
)

In [14]:
prediction = qnx.hugr.cost(
    programs=[ref_hugr], 
    n_shots=[10]
)

In [15]:
import numpy as np

config = qnx.models.HeliosConfig(
    system_name="Helios-1E",
    max_cost=np.ceil(prediction),
    emulator_config=qnx.models.HeliosEmulatorConfig(
        n_qubits=3, 
        simulator=qnx.models.StatevectorSimulator()
    )
)

In [16]:
import numpy as np

result_ref = qnx.start_execute_job(
    programs=[ref_hugr],
    n_shots=[10],
    backend_config=config,
    name=f"Helios-Emulator-{unique_suffix}",
)

In [17]:
qnx.jobs.wait_for(result_ref)
job_result = qnx.jobs.results(result_ref)[0].download_result()

Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.
Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.
Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.
Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.


In [34]:
job_result.results

[QsysShot(entries=[['rus_attempts', 1], ['rus_attempts', 2], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 1], ['rus_attempts', 2], ['rus_attempts', 3], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 1], ['rus_attempts', 3], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 2], ['rus_attempts', 3], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 2], ['rus_attempts', 4], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 1], ['rus_attempts', 2], ['rus_attempts', 3], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 0], ['rus_attempts', 3], ['rus_attempts', 4], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 1], ['rus_attempts', 2], ['rus_attempts', 3], ['rus_attempts', 4], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 2], ['rus_attempts', 3], ['rus_attempts', 4], ['q', 0]]),
 QsysShot(entries=[['rus_attempts', 1], ['rus_attempts', 3], ['rus_attempts', 4], ['q', 0]])]

In [30]:
dir(job_result)


['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_collated_shots_iter',
 'collated_counts',
 'collated_shots',
 'count',
 'index',
 'register_bitstrings',
 'register_counts',
 'results',
 'to_pytket']